In [13]:

import tensorflow as tf
modelo = tf.keras.models.load_model('saved_model/my_model2')

# Check its architecture
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [14]:
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np

def categorizar(url):
  respuesta = requests.get(url)
  img = Image.open(BytesIO(respuesta.content))
  img = np.array(img).astype(float)/255

  img = cv2.resize(img, (224,224))
  prediccion = modelo.predict(img.reshape(-1, 224, 224, 3)) 
  for x in prediccion[0]:
        x = round(x,5)
        print(str(x))
  return np.argmax(prediccion[0], axis=-1)

def categorizar2(path):
    img = Image.open(path)
    img = np.array(img).astype(float)/255
    img = cv2.resize(img, (224,224))
    prediccion = modelo.predict(img.reshape(-1, 224, 224, 3))
    for x in prediccion[0]:
        print(str(x))
    return np.argmax(prediccion[0], axis=-1)

def mostrarCategoria(nombre,window_name):
    print(nombre)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, nombre, (10,450), font, 3, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow(window_name,image)

In [15]:
#0 = carton, 1 = vidrio, 2 = metal ,3 = papel , 4 plastico
url = 'https://img2.rtve.es/a/5897383/?h=320'
prediccion = categorizar (url)
print(prediccion)

0.00044
0.0204
0.00043
0.0
0.97874
4


In [5]:
import os


datos = 'capturas'
if not os.path.exists(datos):
    print('Carpeta creada: ',datos)
    os.makedirs(datos)


In [16]:
count = 0 #para contar las capturas
capture = cv2.VideoCapture(0)
while (capture.isOpened()):
    k = cv2.waitKey(1);
    ret, frame = capture.read()
    cv2.imshow('webCam',frame)
    if (k == ord('s')):  #s para cerrar 
        break
    elif (k == ord('c')): #c para sacar captura
        cv2.imwrite(datos+'/objeto_{}.jpg'.format(count),frame)
        print('Imagen guardada:'+'/objeto_{}.jpg'.format(count))
        ##captura
        path = datos+'/objeto_{}.jpg'.format(count)
        resultado = categorizar2(path)
        print (resultado)
        image = cv2.imread(path)
        window_name = 'image'
        if resultado == 0:
            mostrarCategoria("carton",window_name)
        elif resultado == 1:
            mostrarCategoria("vidrio",window_name)
        elif resultado == 2:
           mostrarCategoria("metal",window_name)
        elif resultado == 3:
           mostrarCategoria("papel",window_name)
        elif resultado == 4:
            mostrarCategoria("plastico",window_name)
        count+=1
        
        


capture.release()
cv2.destroyAllWindows()


Imagen guardada:/objeto_0.jpg
0.005407022
0.09073648
0.17146415
0.019738728
0.71265364
4
plastico


In [16]:
count = 0 #para contar las capturas

# Abrir el vídeo
video = cv2.VideoCapture(2)

# Define los nombres de las clases
class_names = ['carton','vidrio','metal','papel','plastico']

# Determina el tamaño y la posición del texto de la predicción
text_size = cv2.getTextSize(class_names[0], cv2.FONT_HERSHEY_SIMPLEX, 1.0, 3)[0]
text_x = 10
text_y = 30


# Leer el vídeo frame a frame
while video.isOpened():
    # Leer el siguiente frame
    success, frame = video.read()

    # Si el frame se leyó correctamente
    if success:
        count+=1
        cv2.imwrite(datos+'/objeto_{}.jpg'.format(count),frame)
        path = datos+'/objeto_{}.jpg'.format(count)
        img = Image.open(path)
        img = np.array(img).astype(float)/255
        img = cv2.resize(img, (224,224))
        predictions = modelo.predict(img.reshape(-1, 224, 224, 3))
        print(predictions[0])
        max_index = np.argmax(predictions[0])
        max_class = class_names[max_index]
        print(max_index)
        descripcion = str(max_class) +" " + str(predictions[0][max_index])

        # Dibuja el nombre de la clase en el frame
        cv2.putText(frame,descripcion , (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
        
        # Mostrar el frame en una ventana (si lo deseas)
        cv2.imshow('Video', frame)

        # Si se presiona la tecla 'q', detener la reproducción
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Si no se pudo leer el frame, detener la reproducción
    else:
        break

# Liberar los recursos utilizados
video.release()
cv2.destroyAllWindows()


[0.00742051 0.04620994 0.52188975 0.00678747 0.41769242]
2
[0.00867042 0.04461845 0.5010465  0.00799927 0.4376654 ]
2
[0.00904101 0.05704695 0.4789581  0.00983895 0.445115  ]
2
[0.00591941 0.05275447 0.45406944 0.00747026 0.47978643]
4
[0.00589983 0.06061259 0.47229967 0.00686222 0.45432562]
2
[0.00636701 0.06376687 0.4385599  0.00844179 0.48286444]
4
[0.01258484 0.06439556 0.34176108 0.00922454 0.57203406]
4
[0.01153073 0.07325723 0.3871773  0.0091204  0.5189144 ]
4
[0.01002002 0.05997682 0.48606887 0.00666095 0.43727326]
2
[0.00991443 0.05133725 0.5126024  0.00713649 0.4190095 ]
2
[0.00779931 0.04422003 0.5085346  0.00796775 0.4314783 ]
2
[0.00725134 0.05144954 0.4745915  0.00861294 0.4580947 ]
2
[0.00691046 0.05075033 0.45603386 0.00822225 0.47808307]
4
[0.00674444 0.04739128 0.48047632 0.00777879 0.45760918]
2
[0.00319762 0.03884583 0.4064214  0.00700712 0.54452807]
4
[0.01577345 0.078503   0.36254776 0.02589953 0.51727635]
4
[0.08779788 0.1714096  0.54336697 0.00114904 0.19627649]

[0.02161639 0.08257476 0.07665086 0.00667505 0.8124829 ]
4
[0.01185401 0.03568603 0.0445598  0.00494719 0.90295297]
4
[0.0184513  0.04761707 0.03160866 0.00705664 0.89526635]
4
[0.02532207 0.03894185 0.04470961 0.00931471 0.88171184]
4
[0.01371259 0.06853817 0.07673726 0.00853794 0.83247405]
4
[0.01301968 0.04882572 0.06988559 0.00609322 0.86217576]
4
[0.01088567 0.04746143 0.07764444 0.00471993 0.8592885 ]
4
[0.00847912 0.0699193  0.0440953  0.00536757 0.87213874]
4
[0.01577048 0.1068633  0.06367727 0.00882368 0.8048653 ]
4
[0.02563551 0.08669794 0.06833437 0.01053607 0.80879605]
4
[0.0231969  0.08522748 0.06673612 0.01031249 0.81452703]
4
[0.02987553 0.0801047  0.03445332 0.00709552 0.8484709 ]
4
[0.1708447  0.03260069 0.06070996 0.2072762  0.5285685 ]
4
[6.8458617e-01 4.5393992e-04 1.2185360e-02 1.2750562e-01 1.7526892e-01]
0
[0.34719467 0.00410702 0.01194516 0.06224633 0.57450676]
4
[0.01827976 0.0141535  0.00408587 0.799926   0.16355485]
3
[0.07336537 0.00238905 0.01841409 0.79148

[2.8777711e-02 1.6781367e-02 5.2091439e-04 1.0026391e-04 9.5381975e-01]
4
[1.7904483e-02 1.5737079e-02 5.8545248e-04 2.1698381e-04 9.6555603e-01]
4
[1.2992371e-02 1.7925829e-02 6.7075947e-04 3.7189078e-04 9.6803916e-01]
4
[5.0409962e-03 1.7363637e-03 7.7773497e-05 2.0176469e-04 9.9294311e-01]
4
[3.2374926e-02 2.6538361e-02 5.4278143e-04 5.0246087e-04 9.4004148e-01]
4
[4.0005483e-03 5.9366035e-03 2.2819111e-04 8.2107290e-04 9.8901367e-01]
4
[5.2718055e-03 1.0240106e-02 3.2214823e-04 2.1750245e-03 9.8199087e-01]
4
[2.8864495e-03 2.9866526e-02 2.9224675e-04 4.7704374e-04 9.6647769e-01]
4
[3.0119577e-03 5.4782819e-02 3.9892556e-04 8.7591039e-04 9.4093037e-01]
4
[1.8724622e-03 1.7095114e-01 6.9848995e-04 1.0652468e-03 8.2541269e-01]
4
[2.1858250e-03 1.0010371e-01 2.6830254e-04 5.3919328e-04 8.9690298e-01]
4
[6.0917051e-03 8.7504141e-02 3.8811177e-04 4.7027567e-04 9.0554571e-01]
4
[1.2509205e-02 2.4675605e-01 4.3740624e-04 4.9238902e-04 7.3980492e-01]
4
[1.09470505e-02 5.19592524e-01 9.66634